In [1]:
import pandas as pd

In [2]:
file = "data/access-export/derLetzteRest.xlsx"

In [3]:
df = pd.read_excel(file)

In [4]:
# create necessary concept-schemes
ecce_pos,_ = SkosConceptScheme.objects.get_or_create(dc_title="ecce-pos")

In [5]:
pos = set(df['PoS'])

In [6]:
for x in pos:
    temp_variable,_ = SkosConcept.objects.get_or_create(pref_label=x)
    temp_variable.save()
    temp_variable.scheme = [ecce_pos]

In [7]:
# ceate objects
for i, row in df.iterrows():
    temp_object,_ = Token.objects.get_or_create(
        legacy_id=row['ID'],
        spelling=row['spelling'],
        left_context=row['leftcontext'],
        plain_word=row['plainword'],
        comments=row['comments'],
        right_context=row['rightcontext'],
        file=row['file'],
        medial_suffix=row['medialsuffix'],
        final_suffix=row['finalsuffix'],     
    )
    try:
        temp_pos = SkosConcept.objects.get(pref_label=row['PoS'])
        temp_object.pos=temp_pos
        temp_object.save()
    except:
        pass
    try:
        temp_fk = TokenLabel.objects.get(label=row['label'])
        temp_object.label=temp_fk
        temp_object.save()
    except:
        pass
    try:
        temp_fk = Text.objects.get(text=row['textsource'])
        temp_object.text_source=temp_fk
        temp_object.save()
    except:
        pass
    try:
        temp_fk = Cluster.objects.get(consonant=row['cluster'])
        temp_object.cluster=temp_fk
        temp_object.save()
    except:
        pass
    try:
        temp_fk = SchwaPresent.objects.get(spelling=row['spelling2'])
        temp_object.spelling2=temp_fk
        temp_object.save()
    except:
        pass
    try:
        temp_fk = OnSet.objects.get(rightonset=row['rightonset'])
        temp_object.rightonset=temp_fk
        temp_object.save()
    except:
        pass

In [1]:
#for x in Token.objects.all():
    #x.delete()